In [1]:
import time
import pandas as pd
import numpy as np
import openai

import stanza

2022-06-20 19:57:34.122630: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cluster/apps/gcc-6.3.0/openblas-0.2.20-cot3cawsqf4pkxjwzjexaykbwn2ch3ii/lib:/cluster/apps/nss/gcc-6.3.0/python/3.7.4/x86_64/lib64:/cluster/spack/apps/linux-centos7-x86_64/gcc-4.8.5/gcc-6.3.0-sqhtfh32p5gerbkvi5hih7cfvcpmewvj/lib64:/cluster/spack/apps/linux-centos7-x86_64/gcc-4.8.5/gcc-6.3.0-sqhtfh32p5gerbkvi5hih7cfvcpmewvj/lib:/cluster/apps/lsf/10.1/linux2.6-glibc2.3-x86_64/lib
2022-06-20 19:57:34.122685: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
data_path="../../data/"
output_path="../../outputs/"

In [3]:
positives=pd.read_csv(output_path+"positives_ready.csv")

In [4]:
positives=positives.assign(abstract_for_prompt=positives.abstract_for_prompt.fillna(""))
positives=positives.assign(acknowledgments_for_prompt=positives.acknowledgments_for_prompt.fillna(""))

positives=positives.assign(paper_text="Title: "+positives.title_clean+"\n"+positives.abstract_for_prompt+
                          positives.acknowledgments_for_prompt)

In [5]:
nlp = stanza.Pipeline('en', processors={'ner': 'CoNLL03'})

2022-06-20 19:58:14 INFO: Loading these models for language: en (English):
| Processor    | Package  |
---------------------------
| tokenize     | combined |
| pos          | combined |
| lemma        | combined |
| depparse     | combined |
| sentiment    | sstplus  |
| constituency | wsj      |
| ner          | conll03  |

2022-06-20 19:58:14 INFO: Use device: gpu
2022-06-20 19:58:14 INFO: Loading: tokenize
2022-06-20 19:58:19 INFO: Loading: pos
2022-06-20 19:58:19 INFO: Loading: lemma
2022-06-20 19:58:20 INFO: Loading: depparse
2022-06-20 19:58:20 INFO: Loading: sentiment
2022-06-20 19:58:20 INFO: Loading: constituency
2022-06-20 19:58:22 INFO: Loading: ner
2022-06-20 19:58:22 INFO: Done loading processors!


In [6]:
def get_org(x):
    doc=nlp(x)
    orgs=[]
    for sentence in doc.sentences:
        for e in sentence.ents:
            if e.type=='ORG':
                orgs.append(e.text)
    return orgs

In [7]:
positives=positives.assign(organization=positives.paper_text.apply(lambda x:get_org(x)))

In [9]:
positives=positives.assign(len_org=positives.organization.apply(len))

In [11]:
positives=positives.assign(organization_ack=positives.acknowledgments_for_prompt.apply(lambda x:get_org(x)))

In [12]:
positives=positives.assign(organization_abstract=positives.title_abstract_clean.apply(lambda x:get_org(x)))

In [20]:
positives.loc[:,['ID','title','organization_abstract','organization_ack']].to_csv(output_path+"organizations_stanza_conll.csv",index=False)

In [21]:
orgs_ack=positives.loc[:,['ID','organization_ack']]

orgs_ack=orgs_ack.explode('organization_ack')

In [22]:
orgs_abs=positives.loc[:,['ID','organization_abstract']]

orgs_abs=orgs_abs.explode('organization_abstract')

In [23]:
orgs_ack=orgs_ack.dropna()
orgs_abs=orgs_abs.dropna()

In [24]:
orgs_ack_common=orgs_ack.organization_ack.value_counts().reset_index().rename(columns={'index':'name'}).head(400)

In [25]:
pd.set_option('display.max_rows', 400)

In [26]:
orgs_ack_common

,name,organization_ack
0,National Science Foundation,115
1,NSF,106
2,DARPA,96
3,European Union,71
4,National Natural Science Foundation of China,62
5,NIH,54
6,DFG,53
7,National Institutes of Health,52
8,ANR,50
9,IARPA,45


In [27]:
orgs_ack_common=orgs_ack_common.loc[~orgs_ack_common.name.isin(['No organization','Twitter', 'NLP', 'Reddit', 'Facebook',
       'Social Media', 'EU', 'ASR', 'BioNLP', 'AI', 'BioASQ', 'Google','IBM'
       'CLARIN', 'Amazon', 'ACL', 'ERC', 'Microsoft', 'SemEval'])]

In [28]:
orgs_abs_real=orgs_abs.loc[orgs_abs.organization_abstract.isin(orgs_ack_common.name.unique())]

In [29]:
orgs_ack=orgs_ack.rename(columns={'organization_ack':'organization'})
orgs_abs_real=orgs_abs_real.rename(columns={'organization_abstract':'organization'})

In [30]:
organizations=pd.concat([orgs_ack,orgs_abs_real])

In [31]:
organizations=organizations.loc[~organizations.organization.isin(['NLP'])]

In [32]:
organizations=organizations.assign(organization=organizations.organization.str.lstrip("the "))

In [33]:
organizations=organizations.assign(organization=np.where(organizations.organization.isin(['NSF']),
                                                        "Natural Science Foundation",
                                                        organizations.organization))

In [34]:
organizations.to_csv(output_path+"organizations_stanza_conll_clean.csv",index=False)

In [37]:
organizations.organization.value_counts().reset_index().head(400)

,index,organization
0,National Science Foundation,117
1,Natural Science Foundation,108
2,DARPA,100
3,European Union,85
4,Natural Language Processing,71
5,National Natural Science Foundation of China,62
6,COVID,60
7,DFG,55
8,NIH,55
9,CLARIN,55
